In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import math
import random
import scipy

# Load data from Excel file
# data = pd.read_excel('C:/Users/Dear User/Neural Networks/Excercise2/Datasets/classification-iris.xlsx', header=None).values
data = pd.read_excel('C:/Users/Dear User/Neural Networks/Excercise2/Datasets/dataset_diabetic.xlsx', header=None).values
# data = pd.read_excel('C:/Users/Dear User/Neural Networks/Excercise2/Datasets/classification-glass.xlsx', header=None).values

# Shuffle the data
data = shuffle(data, random_state=42)

num_data = data.shape[0] 
num_col = data.shape[1]
num_class = len(np.unique(data[:,num_col-1]))

# Normalize the input data
for ii in range(num_col-1): 
    data[:, ii] = (data[:, ii] / np.max(data[:, ii]) )
# print('data: ', data)    
    
# Split the dataset into a training and testing set
# X_train, X_test,y_train, y_test = train_test_split(data[:num_data, :3], data[:num_data,3], test_size=0.3, random_state=42)

split_ratio_train = 0.7

split_line_number = int(np.shape(data)[0] * split_ratio_train)
X_train = data[:split_line_number, :(num_col-1)]
y_train = data[:split_line_number, num_col-1]
num_train = len(X_train)
             
# other_data = data[split_line_number:, :4]

X_test = data[split_line_number:len(data), :(num_col-1)]
y_test = data[split_line_number:len(data), num_col-1] 
num_test = len(X_test)


# Define the number of input, hidden, and output neurons
input_neurons = X_train.shape[1]

# for iris
l1_neurons = 3
l2_neurons = 12

# for diabet
# l1_neurons = 3
# l2_neurons = 12
output_neurons = num_class  

# Convert labels to one-hot encoding (necessary for multi-class classification)
y_one_hot_train = np.zeros((num_train, num_class))
y_one_hot_train[np.arange(num_train), data[:num_train, num_col-1].astype(int)-1] = 1

y_one_hot_test = np.zeros((num_test, num_class))
y_one_hot_test[np.arange(num_test), data[num_train:, num_col-1].astype(int)-1] = 1


# Initialize the weights with random values in range (-1,1)
np.random.seed(1)
w1 = 2 * np.random.random((input_neurons, l1_neurons)) - 1
w2 = 2 * np.random.random((l1_neurons, l2_neurons)) - 1
w3 = 2 * np.random.random((l2_neurons, output_neurons)) - 1
# print('w1: ', w1, 'w2: ',w2, 'w3: ',w3)

# Initialize the biases with random values in range (-1,1)
b1 = 2 * np.random.random(l1_neurons) - 1
b2 = 2 * np.random.random(l2_neurons) - 1
b3 = 2 * np.random.random(output_neurons) - 1

# 𝛼,𝛽,𝛾,𝛿 = round(random.uniform(-1, 1), 2),round(random.uniform(-1, 1), 2),round(random.uniform(-1, 1), 2),round(random.uniform(-1, 1), 2)

𝛼1,𝛽1,𝛾1,𝛿1 =  np.random.uniform(-1, 1, (l1_neurons, 1)) ,np.random.uniform(-1, 1, (l1_neurons, 1)),np.random.uniform(-1, 1, (l1_neurons, 1)),np.random.uniform(-1, 1, (l1_neurons, 1))
𝛼2,𝛽2,𝛾2,𝛿2 = np.random.uniform(-1, 1, (l2_neurons, 1)),np.random.uniform(-1, 1, (l2_neurons, 1)),np.random.uniform(-1, 1, (l2_neurons, 1)),np.random.uniform(-1, 1, (l2_neurons, 1))
𝛼3,𝛽3,𝛾3,𝛿3 = np.random.uniform(-1, 1, (output_neurons, 1)),np.random.uniform(-1, 1, (output_neurons, 1)),np.random.uniform(-1, 1, (output_neurons, 1)),np.random.uniform(-1, 1, (output_neurons, 1))

# Training parameters
learning_rate = 0.01
epochs = 100  # Train sample by sample

# Regularization parameters
lambda_reg =0.0003

𝛼1_train,𝛽1_train,𝛾1_train,𝛿1_train = np.zeros((epochs,l1_neurons)),np.zeros((epochs,l1_neurons)),np.zeros((epochs,l1_neurons)),np.zeros((epochs,l1_neurons))
𝛼2_train,𝛽2_train,𝛾2_train,𝛿2_train = np.zeros((epochs,l2_neurons)),np.zeros((epochs,l2_neurons)),np.zeros((epochs,l2_neurons)),np.zeros((epochs,l2_neurons))
𝛼3_train,𝛽3_train,𝛾3_train,𝛿3_train = np.zeros((epochs,output_neurons)),np.zeros((epochs,output_neurons)),np.zeros((epochs,output_neurons)),np.zeros((epochs,output_neurons))


mse_train = np.zeros((epochs,num_class))
mse_test = np.zeros((epochs,num_class)) 
CorrectiveMatrix = np.array((num_class,1))

In [ ]:
def 𝑙𝑜𝑔𝑠𝑖𝑔(x):
    return 1/(1 + (scipy.special.expit(-1 * x)))

def 𝑡𝑎𝑛𝑠𝑖𝑔(x):
    return 2/(1+scipy.special.expit(-2*x))-1

def relu(x):
    return np.maximum(0, x)

def 𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(x):
    return x

def sigmoid_derivative(x):
    return (1/(1 + scipy.special.expit(-x))) * (1 - 1/(1 + scipy.special.expit(-x)))
    
def tanh_derivative(x):
    tanh_x = tansig(x)
    return 1 - tanh_x**2

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def pureline_derivative(x):
    return np.ones_like(x)


# 𝐹1(𝑥) = 𝛼 × 𝑙𝑜𝑔𝑠𝑖𝑔(𝑥) + 𝛽 × 𝑡𝑎𝑛𝑠𝑖𝑔(𝑥) + 𝛾 × 𝑅𝑒𝑙𝑢(𝑥) 
def Function2(x,𝛼,𝛽,𝛾): 
    return 𝛼.T * 𝑙𝑜𝑔𝑠𝑖𝑔(x) + 𝛽.T * 𝑡𝑎𝑛𝑠𝑖𝑔(x) + 𝛾.T * relu(x) 

def Function2_derivative(x, 𝛼, 𝛽, 𝛾): 
    # Sum up the derivatives with the corresponding coefficients
    result = 𝛼* sigmoid_derivative(x).T  + 𝛽 * tanh_derivative(x).T + 𝛾 * relu_derivative(x).T 
    return np.diag(result.flatten())


In [ ]:
# Training the MLP for regression
for epoch in range(epochs):
    print('Epoch: {} \t'.format(epoch+1)) 
#     # Shuffle the training data in each epoch
#     shuffle_indices = np.arange(len(X_train))
#     np.random.shuffle(shuffle_indices)
#     X_train = X_train[shuffle_indices]
#     y_train = y_train[shuffle_indices]

    error_data_train = np.zeros((len(X_train),num_class))
    output_data_train = np.zeros(len(X_train))
    for i in range(len(X_train)):
        #-------------------------------- Feed Forward -------------------------------------
        input_layer = X_train[i:i+1]
        
        net1 = np.dot(input_layer, w1) +b1.reshape(1,-1)   # net1 = x * w1 +b1
        
        o1   = Function2(net1,𝛼1,𝛽1,𝛾1) #  o1 = f(net1)
        
        net2 = np.dot(o1, w2) + b2.reshape(1,-1)  # net2 = o1 * w2 +b2

        o2   = Function2(net2,𝛼2,𝛽2,𝛾2) #  o2 = f(net2)
        
        net3 = np.dot(o2, w3) + b3.reshape(1,-1)    # net3 = o2 * w3 +b3
        
        o3   = Function2(net3,𝛼3,𝛽3,𝛾3) #  o3 = f(net3)
        
        output_data_train[i] = np.argmax(o3)+1
        
        #-------------------------------- Backpropagation ----------------------------------- 
#         output_layer_error = y_train[i:i+1] - output_data_train[i]
        output_layer_error = ((y_one_hot_train[i:i+1]).flatten() - o3.flatten()).reshape(1,-1) 
#         output_layer_error =  output_layer_error * 0.1

        # dE/da = dE/de * de/do3 * do3/da => o3=F(net3,a)= a*logsig + ... => dE/da = e * -1 * logsig(net3)
        diag_𝑙𝑜𝑔𝑠𝑖𝑔_net3 = np.array(np.diag(𝑙𝑜𝑔𝑠𝑖𝑔(net3.flatten())))
        diag_𝑡𝑎𝑛𝑠𝑖𝑔_net3 = np.array(np.diag(𝑡𝑎𝑛𝑠𝑖𝑔(net3.flatten())))
        diag_relu_net3 = np.array(np.diag(relu(net3.flatten()))) 
        diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net3 = np.array(np.diag(𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(net3.flatten())))
        
        𝛼3 = 𝛼3 + learning_rate * (output_layer_error @ diag_𝑙𝑜𝑔𝑠𝑖𝑔_net3).T - lambda_reg * 𝛼3
        𝛽3 = 𝛽3 + learning_rate * (output_layer_error @ diag_𝑡𝑎𝑛𝑠𝑖𝑔_net3).T - lambda_reg * 𝛽3
        𝛾3 = 𝛾3 + learning_rate * (output_layer_error @ diag_relu_net3).T - lambda_reg * 𝛾3
        𝛿3 = 𝛿3 + learning_rate * (output_layer_error @ diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net3).T - lambda_reg * 𝛿3
        
        # update w3
        w3_old = w3
        fprim_net3 = np.array(Function2_derivative(net3,𝛼3,𝛽3,𝛾3))
        w3 = w3 +  learning_rate * o2.T @ output_layer_error @ fprim_net3 - lambda_reg * w3
        b3 = b3 +  learning_rate *        output_layer_error @ fprim_net3 * 1 -lambda_reg*b3
        
        diag_𝑙𝑜𝑔𝑠𝑖𝑔_net2 = np.array(np.diag(𝑙𝑜𝑔𝑠𝑖𝑔(net2.flatten()))) 
        diag_𝑡𝑎𝑛𝑠𝑖𝑔_net2= np.array(np.diag(𝑡𝑎𝑛𝑠𝑖𝑔(net2.flatten())))  
        diag_relu_net2 = np.array(np.diag(relu(net2.flatten()))) 
        diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net2 = np.array(np.diag(𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(net2.flatten())))  
        
        # dE/da = dE/de * de/do3 * do3/dnet3 * dnet3/do2 * do2/da2 
        # => o2=F(net2,a)= a*logsig + ... => dE/da = e * -1 *F'(net3) * w3_old * logsig(net2)
        𝛼2 = 𝛼2 + learning_rate *  diag_𝑙𝑜𝑔𝑠𝑖𝑔_net2 @ w3_old @ fprim_net3 @ output_layer_error.T - lambda_reg * 𝛼2
        𝛽2 = 𝛽2 + learning_rate *  diag_𝑡𝑎𝑛𝑠𝑖𝑔_net2 @  w3_old @ fprim_net3 @ output_layer_error.T - lambda_reg * 𝛽2
        𝛾2 = 𝛾2 + learning_rate *  diag_relu_net2 @ w3_old @ fprim_net3 @ output_layer_error.T - lambda_reg * 𝛾2
        𝛿2 = 𝛿2 + learning_rate *  diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net2 @ w3_old @ fprim_net3 @ output_layer_error.T - lambda_reg * 𝛿2
    
        # update w2
        w2_old = w2
        # Create a diagonal matrix from the 1D array
        fprim_net2 = np.array(Function2_derivative(net2,𝛼2,𝛽2,𝛾2))
#       w2 = w2 + learning_rate * output_layer_error * w3_old.dot(np.dot(np.dot(o1.T,fprim_net2),fprim_net3))
        w2 = w2 + learning_rate * o1.T @ output_layer_error @ fprim_net3 @ w3.T @ fprim_net2 - lambda_reg * w2
        b2 = b2 + learning_rate *        output_layer_error @ fprim_net3 @ w3.T @ fprim_net2 - lambda_reg * b2
        
    
        # dE/da = dE/de * de/do3 * do3/dnet3 * dnet3/do2 * do2/dnet2 *dnet2/do1 * do1/da1
        # => o1=F(net1,a)= a*logsig + ... => dE/da = e * -1 *F'(net3) * w3 * F'(net2) *w2 * logsig(net1)
        diag_𝑙𝑜𝑔𝑠𝑖𝑔_net1 = np.array(np.diag(𝑙𝑜𝑔𝑠𝑖𝑔(net1.flatten())))
        diag_𝑡𝑎𝑛𝑠𝑖𝑔_net1= np.array(np.diag(𝑡𝑎𝑛𝑠𝑖𝑔(net1.flatten())))
        diag_relu_net1 = np.array(np.diag(relu(net1.flatten())))
        diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net1= np.array(np.diag(𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(net1.flatten())))
        
        𝛼1 = 𝛼1 + learning_rate * diag_𝑙𝑜𝑔𝑠𝑖𝑔_net1 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 @ output_layer_error.T - lambda_reg * 𝛼1
        𝛽1 = 𝛽1 + learning_rate * diag_𝑡𝑎𝑛𝑠𝑖𝑔_net1 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 @ output_layer_error.T - lambda_reg * 𝛽1
        𝛾1 = 𝛾1 + learning_rate * diag_relu_net1 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 @ output_layer_error.T - lambda_reg * 𝛾1 
        𝛿1 = 𝛿1 + learning_rate * diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net1 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 @ output_layer_error.T - lambda_reg * 𝛿1 
        
        # update w1        
        fprim_net1 = np.array(Function2_derivative(net1,𝛼1,𝛽1,𝛾1))
        w1 = w1 + learning_rate * input_layer.T @ output_layer_error @ fprim_net3 @ w3_old.T @ fprim_net2 @ w2_old.T @ fprim_net1 - lambda_reg * w1
        b1 = b1 + learning_rate *                 output_layer_error @ fprim_net3 @ w3_old.T @ fprim_net2 @ w2_old.T @ fprim_net1 * 1  -lambda_reg*b1
    
        error_data_train[i] = output_layer_error
    
    mse_train[epoch] = np.mean(error_data_train ** 2, axis=0)
    𝛼1_train[epoch] = 𝛼1.T
    𝛽1_train[epoch] = 𝛽1.T
    𝛾1_train[epoch] = 𝛾1.T
    𝛿1_train[epoch] = 𝛿1.T
    
    𝛼2_train[epoch] = 𝛼2.T
    𝛽2_train[epoch] = 𝛽2.T
    𝛾2_train[epoch] = 𝛾2.T
    𝛿2_train[epoch] = 𝛿2.T
    
    𝛼3_train[epoch] = 𝛼3.T
    𝛽3_train[epoch] = 𝛽3.T
    𝛾3_train[epoch] = 𝛾3.T
    𝛿3_train[epoch] = 𝛿3.T
    plt.figure(1, figsize=(10, 8))

    # Plotting the first subplot
    plt.subplot(2, 2, 1)
    plt.plot(data[:num_train, num_col-1], '-sr', label='Target Data')
    plt.plot(output_data_train, '-*b',label='Predicted Data')
    plt.xlabel('Train Data')
    plt.ylabel('Output')
    plt.legend()
    
    # Plotting the second subplot
    plt.subplot(2, 2, 2)
    labels = [f'MSE{i}' for i in range(1, num_class + 1)]
    plt.semilogy(np.arange(1, epoch + 1), mse_train[:epoch], label=labels)

    plt.xlabel('Epoch')
    plt.ylabel('MSE Train')
    plt.legend()

    # Adjust layout for better spacing
    plt.tight_layout()

    # Show the plots
    plt.show()
    
    print('MSE_train:  ', mse_train[epoch])

    print("\n\033[1;m" + "*" * 125)

#     if epoch % 100 == 0:
#         print(f"Epoch {epoch}: Total Error = {total_error[epoch]}")

print("****************************** Training completed *******************************")


In [ ]:
# Testing the trained MLP for regression
total_accuracy = 0 
error_data_test = np.zeros((len(X_test),num_class))
output_data_test = np.zeros(len(X_test))
for i in range(len(X_test)):
    input_layer = X_test[i:i+1]

    net1 = np.dot(input_layer, w1)+b1   # net1 = x * w1+b1
    o1   = Function2(net1,𝛼1,𝛽1,𝛾1) #  o1 = f(net1)
      
    net2 = np.dot(o1, w2)+b2 # net2 = o1 * w2+b2
    o2   = Function2(net2,𝛼2,𝛽2,𝛾2) #  o2 = f(net2)
        
    net3 = np.dot(o2, w3)+b3  # net3 = o2 * w3+b3
    o3   = Function2(net3,𝛼3,𝛽3,𝛾3) #  o3 = f(net3)
    
    output_data_test[i] = np.argmax(o3)+1
#     error_data_test[i] = y_test[i] - output_data_test[i]
    error_data_test[i] = (y_one_hot_test[i:i+1]).flatten() - o3.flatten()

mse_test[epoch] = np.mean(error_data_test ** 2, axis=0)
plt.figure(1, figsize=(8, 6))
plt.subplot(2, 2, 3)
plt.plot(data[num_train:, num_col-1], '-sr')
plt.plot(output_data_test, '-*b')
plt.xlabel('Test Data')
plt.ylabel('Output')

print('Epoch: '.format(epoch+1))
print('MSE_train:   ',mse_train[epoch],'\t, ','MSE_Test:  ', mse_test[epoch])

plt.tight_layout()
plt.show()


mse_train_result = mse_train[-1]
mse_test_result = mse_test[-1]

print("Final MSE on Train Data:", mse_train_result)
print("Final MSE on Test Data:", mse_test_result)

train_accuracy = np.mean(output_data_train == y_train) 
print(f"Accuracy on the train set: {train_accuracy * 100}%")

test_accuracy = np.mean(output_data_test == y_test) 
print(f"Accuracy on the test set: {test_accuracy * 100}%")


In [ ]:

# Assuming you have converted regression predictions to classes
predicted_classes = output_data_train 
actual_classes = data[:num_train,num_col-1] 

# Define the number of classes

# Create a confusion matrix-like matrix
confusion_matrix = np.zeros((num_class, num_class))

# Fill the confusion matrix
for actual, predicted in zip(actual_classes, predicted_classes):
    confusion_matrix[actual.astype(int)-1][predicted.astype(int)-1] += 1

# Plot the confusion matrix
plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Train Confusion Matrix ')
plt.colorbar()

# Annotate the plot with numbers
for i in range(num_class):
    for j in range(num_class):
        plt.text(j, i, str(int(confusion_matrix[i, j])), fontsize=12, ha='center', va='center')  # Corrected

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(np.arange(num_class), np.arange(1, num_class + 1))
plt.yticks(np.arange(num_class), np.arange(1, num_class + 1))

plt.show()


In [ ]:

# Assuming you have converted regression predictions to classes
predicted_classes = output_data_test 
actual_classes = data[num_train:,num_col-1] 

# Define the number of classes

# Create a confusion matrix-like matrix
confusion_matrix = np.zeros((num_class, num_class))

# Fill the confusion matrix
for actual, predicted in zip(actual_classes, predicted_classes):
    confusion_matrix[actual.astype(int)-1][predicted.astype(int)-1] += 1

# Plot the confusion matrix
plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Test Confusion Matrix ')
plt.colorbar()

# Annotate the plot with numbers
for i in range(num_class):
    for j in range(num_class):
        plt.text(j, i, str(int(confusion_matrix[i, j])), fontsize=12, ha='center', va='center')  # Corrected
# Adjust ticks and labels to start from 1
        
plt.xlabel('Predicted')
plt.ylabel('Actual')
# plt.xticks(np.arange(num_class))
# plt.yticks(np.arange(num_class))
plt.xticks(np.arange(num_class), np.arange(1, num_class + 1))
plt.yticks(np.arange(num_class), np.arange(1, num_class + 1))

plt.show()



In [ ]:
from sklearn.linear_model import LinearRegression 

# Plotting regression 
plt.figure(figsize=(8, 6)) 
plt.scatter(y_train, output_data_train, color='blue', marker='o', label='Predicted vs. Actual') 

# Fitting linear regression line 
regression = LinearRegression() 
regression.fit(y_train.reshape(-1, 1), output_data_train.reshape(-1, 1))  # Reshape to 2D arrays

# Plotting Y=T line
plt.plot(y_train, y_train, color='black',  label='Y=T') 

# Plotting the fitted line (regression line) with dashed style
plt.plot(y_train, regression.predict(y_train.reshape(-1, 1)), color='red',  label='Fit') 

plt.title('Regression Train') 
plt.xlabel('Target Values') 
plt.ylabel('Predicted Outputs') 
plt.legend() 
plt.grid(True) 
plt.show()


In [ ]:
from sklearn.linear_model import LinearRegression 

# Plotting regression 
plt.figure(figsize=(8, 6)) 
plt.scatter(y_test, output_data_test, color='blue', marker='o', label='Predicted vs. Actual') 

# Fitting linear regression line 
regression = LinearRegression() 
regression.fit(y_test.reshape(-1, 1), output_data_test.reshape(-1, 1))  # Reshape to 2D arrays

# Plotting Y=T line
plt.plot(y_test, y_test, color='black',  label='Y=T') 

# Plotting the fitted line (regression line) with dashed style
plt.plot(y_test, regression.predict(y_test.reshape(-1, 1)), color='red',  label='Fit') 

plt.title('Regression Test') 
plt.xlabel('Target Values') 
plt.ylabel('Predicted Outputs') 
plt.legend() 
plt.grid(True) 
plt.show()


In [ ]:
plt.figure(figsize=(15, 5)) 

# Plotting 𝛼1_train for each neuron in the same row
for i in range(l1_neurons):
    plt.subplot(1, l1_neurons, i+1)
    plt.plot(np.arange(0, epoch + 1), 𝛼1_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('alpha1')
    plt.title(f'Layer 1, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()
plt.show()


# Plotting 𝛽1_train for each neuron in the same row
plt.figure(figsize=(15, 5))
for i in range(l1_neurons):
    plt.subplot(1, l1_neurons, i+1)
    plt.plot(np.arange(0, epoch + 1), 𝛽1_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('beta1')
    plt.title(f'Layer 1, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout() 
plt.show()


# Plotting 𝛾1_train for each neuron in the same row
plt.figure(figsize=(15, 5))
for i in range(l1_neurons):
    plt.subplot(1, l1_neurons, i+1)
    plt.plot(np.arange(0, epoch + 1), 𝛾1_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('gama1')
    plt.title(f'Layer 1, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout() 
plt.show()


# Plotting 𝛿1_train for each neuron in the same row
plt.figure(figsize=(15, 5))
for i in range(l1_neurons):
    plt.subplot(1, l1_neurons, i+1)
    plt.plot(np.arange(0, epoch + 1), 𝛿1_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('delta1')
    plt.title(f'Layer 1, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout() 
plt.show()

In [ ]:
plt.figure(figsize=(15,30)) 

# Number of rows
num_rows = l2_neurons // 3 + (l2_neurons % 3 > 0)  # To ensure all neurons are covered

# Plotting 𝛼2_train for each neuron in the same row
for i in range(l2_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3) 
    plt.plot(np.arange(0, epoch + 1), 𝛼2_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('alpha2')
    plt.title(f'Layer 2, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout()
plt.show()


# Plotting 𝛽2_train for each neuron in the same row
plt.figure(figsize=(15, 30))
for i in range(l2_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝛽2_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('beta2')
    plt.title(f'Layer 2, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout() 
plt.show()


# Plotting 𝛾2_train for each neuron in the same row
plt.figure(figsize=(15, 30))
for i in range(l2_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝛾2_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('gama2')
    plt.title(f'Layer 2, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout() 
plt.show()


# Plotting 𝛿2_train for each neuron in the same row
plt.figure(figsize=(15, 30))
for i in range(l2_neurons):
    plt.subplot(num_rows, 3, i % 3 + 1 + (i // 3) * 3)
    plt.plot(np.arange(0, epoch + 1), 𝛿2_train[:, i], label=f'Neuron {i+1}')
    plt.xlabel('Epoch')
    plt.ylabel('delta2')
    plt.title(f'Layer 2, Neuron {i+1}')
    plt.legend()

# Adjust layout
plt.tight_layout() 
plt.show()